In [ ]:
from datetime import datetime, timedelta
import requests
import pandas as pd

def get_videos_info(usernames, access_token, fields, start_date, end_date, max_count=100):
    endpoint = "https://open.tiktokapis.com/v2/research/video/query/"
    headers = {"Authorization": f"Bearer {access_token}", "Content-Type": "application/json"}
    query_fields = fields.split(',')
    videos_df = pd.DataFrame(columns=query_fields)
    start_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")
    delta = timedelta(days=30)
    session = requests.Session()  # Using session for improved performance

    for username in usernames:
        current_start_date = start_date
        while current_start_date < end_date:
            current_end_date = min(current_start_date + delta, end_date)
            query_body = {
                "query": {
                    "and": [{"operation": "EQ", "field_name": "username", "field_values": [username]}]
                },
                "start_date": current_start_date.strftime("%Y%m%d"),
                "end_date": current_end_date.strftime("%Y%m%d"),
                "max_count": max_count
            }

            response = session.post(f"{endpoint}?fields={','.join(query_fields)}", headers=headers, json=query_body)
            if response.status_code == 200:
                data = response.json().get("data", {})
                videos = data.get("videos", [])
                videos_df = pd.concat([videos_df, pd.DataFrame(videos)], ignore_index=True)
            elif response.status_code == 429:
                print("Rate limit exceeded, sleeping for 60 seconds.")
                time.sleep(60)  # Handling rate limiting
                continue  # Consider whether to retry the last request
            else:
                print(f"Error fetching videos for user {username}: {response.status_code}", response.json())
            current_start_date += delta + timedelta(days=1)
    
    return videos_df